In [1]:
import mne
from mne.datasets import eegbci
import numpy as np
from sklearn.pipeline import Pipeline
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis as LDA
from sklearn.model_selection import cross_val_score, StratifiedKFold
from sklearn.multiclass import OneVsRestClassifier
from mne.decoding import CSP

### 1. Setup and Configuration

This section defines the parameters for our analysis. We specify the subject ID and the corresponding experimental run numbers for the different motor imagery tasks as detailed on the PhysioNet dataset description page.

In [2]:
subject_id = 1
runs_lr = [4, 8, 12] # right or left hand fists data
runs_feet = [6, 10, 14] # feet or both fists data

## 2. Data Loading

Here, we use the `mne.datasets.eegbci.load_data` function to automatically download the required `.edf` files from the public PhysioNet repository. We are downloading two distinct sets of experiments: one for left vs. right hand imagery and another that includes feet imagery.

In [3]:
print("Downloading data...")
fnames_lr = eegbci.load_data(subject_id, runs = runs_lr, update_path = True)
print("Download complete.")

Download complete.


In [4]:
print("Downloading data...")
fnames_feet = eegbci.load_data(subject_id, runs = runs_feet, update_path = True)
print("Download complete.")

Download complete.


## 3. Reading and Combining Raw Data

The downloaded `.edf` files are loaded into MNE's core data structure, the `Raw` object. Since each task (e.g., left/right fist) consists of multiple recording sessions (runs), we concatenate them to create a single, continuous data stream for each task type.

In [5]:
raws_lr = [mne.io.read_raw_edf(f, preload = True) for f in fnames_lr]

Extracting EDF parameters from /Users/Mohammad/mne_data/MNE-eegbci-data/files/eegmmidb/1.0.0/S001/S001R04.edf...
EDF file detected
Setting channel info structure...
Creating raw.info structure...
Reading 0 ... 19999  =      0.000 ...   124.994 secs...
Extracting EDF parameters from /Users/Mohammad/mne_data/MNE-eegbci-data/files/eegmmidb/1.0.0/S001/S001R08.edf...
EDF file detected
Setting channel info structure...
Creating raw.info structure...
Reading 0 ... 19999  =      0.000 ...   124.994 secs...
Extracting EDF parameters from /Users/Mohammad/mne_data/MNE-eegbci-data/files/eegmmidb/1.0.0/S001/S001R12.edf...
EDF file detected
Setting channel info structure...
Creating raw.info structure...
Reading 0 ... 19999  =      0.000 ...   124.994 secs...


In [6]:
raws_feet = [mne.io.read_raw_edf(f, preload = True) for f in fnames_feet]

Extracting EDF parameters from /Users/Mohammad/mne_data/MNE-eegbci-data/files/eegmmidb/1.0.0/S001/S001R06.edf...
EDF file detected
Setting channel info structure...
Creating raw.info structure...
Reading 0 ... 19999  =      0.000 ...   124.994 secs...
Extracting EDF parameters from /Users/Mohammad/mne_data/MNE-eegbci-data/files/eegmmidb/1.0.0/S001/S001R10.edf...
EDF file detected
Setting channel info structure...
Creating raw.info structure...
Reading 0 ... 19999  =      0.000 ...   124.994 secs...
Extracting EDF parameters from /Users/Mohammad/mne_data/MNE-eegbci-data/files/eegmmidb/1.0.0/S001/S001R14.edf...
EDF file detected
Setting channel info structure...
Creating raw.info structure...
Reading 0 ... 19999  =      0.000 ...   124.994 secs...


In [7]:
## Concatenate the list of Raw objects for each task into a single, continuous Raw object.
raw_lr = mne.concatenate_raws(raws_lr)
raw_feet = mne.concatenate_raws(raws_feet)

## 4. Signal Preprocessing and Epoching

This is the most critical data cleaning and preparation stage.

### Signal Filtering
Raw EEG data is noisy. We apply two main filters:
1.  **Band-Pass Filter (8-35 Hz):** We isolate the frequency bands most associated with motor control and imagery, known as the *mu* (μ) and *beta* (β) rhythms. This removes slow signal drifts and high-frequency noise.
2.  **Notch Filter (50 Hz):** This specifically targets and removes electrical noise from the power grid, which is a common and powerful source of interference.

### Epoching
We slice the continuous signal into discrete time windows called **epochs** or **trials**. Each epoch is locked to a specific event (e.g., the cue to imagine moving the left fist). By creating these labeled trials, we transform the data into a format suitable for supervised machine learning. We use a time window from -1s to +4s to also capture the brain's preparatory activity before the cue.

In [12]:
def process_and_epoch(raw, event_id):
    raw.filter(l_freq=8., h_freq=35.)
    raw.notch_filter(freqs=50)
    
    # Extract events from annotations. 'T1' and 'T2' are markers in the data
    # corresponding to the start of different tasks.
    events, _ = mne.events_from_annotations(raw, event_id={'T1': 1, 'T2': 2})

    # starting 1 second before the cue to capture preparatory brain activity.
    epochs = mne.Epochs(raw, events, event_id, tmin=-1., tmax=4., preload=True, baseline=None, picks='eeg')
    return epochs

In [13]:
epochs_lr = process_and_epoch(raw_lr, event_id={'left_fist': 1, 'right_fist': 2})
epochs_feet = process_and_epoch(raw_feet, event_id={'feet': 2})

Filtering raw data in 3 contiguous segments
Setting up band-pass filter from 8 - 35 Hz

FIR filter parameters
---------------------
Designing a one-pass, zero-phase, non-causal bandpass filter:
- Windowed time-domain design (firwin) method
- Hamming window with 0.0194 passband ripple and 53 dB stopband attenuation
- Lower passband edge: 8.00
- Lower transition bandwidth: 2.00 Hz (-6 dB cutoff frequency: 7.00 Hz)
- Upper passband edge: 35.00 Hz
- Upper transition bandwidth: 8.75 Hz (-6 dB cutoff frequency: 39.38 Hz)
- Filter length: 265 samples (1.656 s)

Filtering raw data in 3 contiguous segments
Setting up band-stop filter from 49 - 51 Hz

FIR filter parameters
---------------------
Designing a one-pass, zero-phase, non-causal bandstop filter:
- Windowed time-domain design (firwin) method
- Hamming window with 0.0194 passband ripple and 53 dB stopband attenuation
- Lower passband edge: 49.38
- Lower transition bandwidth: 0.50 Hz (-6 dB cutoff frequency: 49.12 Hz)
- Upper passband edg

In [14]:
# Combine all processed epochs from all tasks into a single object for the classifier.
epochs = mne.concatenate_epochs([epochs_lr, epochs_feet])

Not setting metadata
69 matching events found
No baseline correction applied


/var/folders/1s/bkqkh8453p79rr15mcq_bm4c0000gp/T/ipykernel_55915/4130775407.py:2: RuntimeWarning: Concatenation of Annotations within Epochs is not supported yet. All annotations will be dropped.
  epochs = mne.concatenate_epochs([epochs_lr, epochs_feet])


## 5. Feature and Label Preparation

We extract the final processed data and corresponding labels from the `Epochs` object. This prepares them for direct use with scikit-learn's machine learning models. The data is now a 3D NumPy array (`trials x channels x timepoints`), and the labels are a 1D vector.

In [15]:
data = epochs.get_data() # The data is a 3D numpy array: (n_epochs, n_channels, n_times).
labels = epochs.events[:, -1]

## 6. Machine Learning Pipeline: CSP + LDA

To classify the high-dimensional EEG epochs, we build a pipeline that first extracts meaningful features and then uses a simple classifier.

### Common Spatial Patterns (CSP)
CSP is a feature extraction algorithm that finds spatial filters $W$ to maximize the variance for one class while minimizing it for another. This is ideal for discriminating between motor imagery tasks. The optimization problem it solves is maximizing the Rayleigh quotient:

$$J(w) = \frac{w^T \Sigma_1 w}{w^T \Sigma_2 w}$$

- $w$ is the spatial filter (a set of weights for each channel).
- $\Sigma_1$ is the covariance matrix of the EEG data for the first class (e.g., left fist).
- $\Sigma_2$ is the covariance matrix of the EEG data for the second class (e.g., right fist).

### Linear Discriminant Analysis (LDA)
LDA is a linear classifier that projects the features from CSP onto a line to maximize the separation between the classes. The decision function for a new sample $x$ is:

$$y(x) = w^T x + w_0$$

### One-vs-Rest (OvR) Strategy
Since CSP and LDA are inherently binary, we use the One-vs-Rest (OvR) strategy to handle our 3-class problem. It trains $K$ separate classifiers (where $K=3$ here). For a new sample $x$, the final prediction is the class $k$ whose classifier reports the highest confidence score:

$$\text{class} = \arg\max_{k \in \{1, ..., K\}} f_k(x)$$

In [16]:
csp = CSP(n_components=4, reg=None, log=True) # n_components=4 means we are extracting the 4 most discriminative spatial patterns.
lda = LDA()
ovr_pipeline = OneVsRestClassifier(Pipeline([('CSP', csp), ('LDA', lda)]))

In [17]:
# Set up the cross-validation strategy. StratifiedKFold ensures each fold
# has a representative balance of all three classes.
cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)
scores = cross_val_score(ovr_pipeline, data, labels, cv=cv)

Computing rank from data with rank=None
    Using tolerance 0.00036 (2.2e-16 eps * 64 dim * 2.6e+10  max singular value)
    Estimated rank (data): 64
    data: rank 64 computed from 64 data channels with 0 projectors
Reducing data rank from 64 -> 64
Estimating class=0 covariance using EMPIRICAL
Done.
Estimating class=1 covariance using EMPIRICAL
Done.
Computing rank from data with rank=None
    Using tolerance 0.00037 (2.2e-16 eps * 64 dim * 2.6e+10  max singular value)
    Estimated rank (data): 64
    data: rank 64 computed from 64 data channels with 0 projectors
Reducing data rank from 64 -> 64
Estimating class=0 covariance using EMPIRICAL
Done.
Estimating class=1 covariance using EMPIRICAL
Done.
Computing rank from data with rank=None
    Using tolerance 0.00037 (2.2e-16 eps * 64 dim * 2.6e+10  max singular value)
    Estimated rank (data): 64
    data: rank 64 computed from 64 data channels with 0 projectors
Reducing data rank from 64 -> 64
Estimating class=0 covariance using EMP

In [18]:
print("\n--- 3-Class Cross-Validation Accuracy ---")
print(f"Scores for each fold: {scores}")
print(f"Mean Accuracy: {np.mean(scores):.4f}")
print(f"Standard Deviation: {np.std(scores):.4f}")


--- 3-Class Cross-Validation Accuracy ---
Scores for each fold: [0.57142857 0.35714286 0.64285714 0.64285714 0.92307692]
Mean Accuracy: 0.6275
Standard Deviation: 0.1811
